In [52]:
# I need to read in the pdf, unfortunately, its not a real table so pdfpluber read table wont work 
# Just going to read in for now and use regex to clean as much as needed
# Once its clean enough I will convert it using pandas

In [53]:
import pdfplumber
import pandas
import re

In [54]:
# Regex rules
single_letter = re.compile(r'^[A-Za-z]$')
single_letter_inside = re.compile(r'\b[A-Za-z]\b')
date_year_month = re.compile(r'^(\d{4}-\d{2}-)')
date_day = re.compile(r'^\d{2}')
reference_pattern = re.compile(r'[A-Za-z]{2,5}\d+')
amount_pattern = re.compile(r'[+-]\$[\d,]+\.\d{2}')
balance_pattern = re.compile(r'\$[\d,]+\.\d{2}')

In [55]:
transactions = []
pdf_text = []

In [56]:
test_pdf = "./FakeStatements/Demo Bank Statement - Robert.pdf"

In [57]:
with pdfplumber.open(test_pdf) as pdf:
    for page in pdf.pages:
        pdf_text.extend(page.extract_text().splitlines())
        
pdf_text

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Cannot set gray stroke color because /'P13' is an invalid float value
Cannot set gray non-stroke color because /'P13' is an invalid float value


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


['9/4/25, 9:49 AM Demo Bank Statement - Robert',
 '🚨 FICTIONAL DEMO DOCUMENT - NOT A REAL BANK STATEMENT 🚨',
 'This is for testing, educational, or demonstration purposes only',
 'FB FakeBank Demo',
 'Fictional Financial Institution - Demo Purposes Only',
 'Account Holder Account Details',
 'Robert Demo Sample Account Number: ****-****-3456',
 '123 Test Street Account Type: Demo Premium',
 'Checking',
 'Demo City, ST 12345',
 'O',
 'Statement Period: Aug 1 - Aug 31, 2025',
 'M',
 'Y',
 'Beginning EBalance Total Deposits',
 '$15,750.00 L +$4,250.00',
 'D',
 'N',
 'Total Withdrawals Ending Balance',
 '-$3,875.O25 $16,124.75',
 'Transaction History',
 'Date Description Reference Amount Balance',
 '2025-08-',
 'Opening Balance - - $15,750.00',
 '01',
 '2025-08-',
 'Social Security Deposit SS789012 +$1,850.00 $17,600.00',
 '01',
 '2025-08- Pension Deposit - Demo',
 'PN445566 +$2,400.00 $20,000.00',
 '03 Retirement Fund',
 'file:///C:/Users/aaron/Downloads/family_bank_statement_4.html 1/3',


In [58]:
#Im pretty sure there are random letters in white text on pdf so we need to clean those out

In [59]:
cleaned_text=[]
for line in pdf_text:
    new_line = single_letter_inside.sub('', line).strip()
    if new_line:
        cleaned_text.append(new_line)
pdf_text = cleaned_text

In [60]:
pdf_text

['9/4/25, 9:49 AM Demo Bank Statement - Robert',
 '🚨 FICTIONAL DEMO DOCUMENT - NOT  REAL BANK STATEMENT 🚨',
 'This is for testing, educational, or demonstration purposes only',
 'FB FakeBank Demo',
 'Fictional Financial Institution - Demo Purposes Only',
 'Account Holder Account Details',
 'Robert Demo Sample Account Number: ****-****-3456',
 '123 Test Street Account Type: Demo Premium',
 'Checking',
 'Demo City, ST 12345',
 'Statement Period: Aug 1 - Aug 31, 2025',
 'Beginning EBalance Total Deposits',
 '$15,750.00  +$4,250.00',
 'Total Withdrawals Ending Balance',
 '-$3,875.O25 $16,124.75',
 'Transaction History',
 'Date Description Reference Amount Balance',
 '2025-08-',
 'Opening Balance - - $15,750.00',
 '01',
 '2025-08-',
 'Social Security Deposit SS789012 +$1,850.00 $17,600.00',
 '01',
 '2025-08- Pension Deposit - Demo',
 'PN445566 +$2,400.00 $20,000.00',
 '03 Retirement Fund',
 'file:///:/Users/aaron/Downloads/family_bank_statement_4.html 1/3',
 '9/4/25, 9:49 AM Demo Bank State

In [61]:
#Clean enough for now

In [62]:
Date = ""
Description = ""
Reference = ""
Amount = ""
Balance = ""

start_processing = False

for line in pdf_text:
    # check if we've reached the marker line
    if not start_processing:
        if "transaction history" in line.lower():
            start_processing = True
        continue  # skip until found
    date_match = date_year_month.match(line)
    if date_match:
        Date = date_match.group()
        line = date_year_month.sub("",line).strip()
        Description = line
        # print(f'Date: {Date}')
        # print(f'Description: {Description}')
        
        # so i need a for loop or while loop here after it finds the date to check the next 2 lines
        
        
    ref_match = reference_pattern.search(line)
    if ref_match:
        Reference = ref_match.group()
        # print(f'Ref: {Reference}')
        line = reference_pattern.sub("", line).strip()
        # print(line)
    amt_match = amount_pattern.search(line)
    if amt_match:
        Amount = amt_match.group()
        # print(f'AMT: {Amount}')
        line = amount_pattern.sub("", line).strip()
        # print(line)
    bal_match = balance_pattern.search(line)
    if bal_match:
        Balance = bal_match.group()
        # print(f'BAL: {Balance}')
        line = balance_pattern.sub("",line).strip()
        # print(line)
        if line:
            Description = line
    day_match = date_day.search(line)
    if day_match:
        Date += day_match.group()
        # print(f'Date: {Date}')
        line = date_day.sub("",line)
        # print(line)
        Description += line
        # print(f'Description: {Description}')

        print(f'Date: {Date}')
        print(f'Description: {Description}')
        print(f'Ref: {Reference}')
        print(f'AMT: {Amount}')
        print(f'BAL: {Balance}')
        print("\n")
        
        ##Honestly at this point i could just append to pd dataframe after this line
    

Date: 2025-08-01
Description: Opening Balance - -
Ref: 
AMT: 
BAL: $15,750.00


Date: 2025-08-01
Description: Social Security Deposit
Ref: SS789012
AMT: +$1,850.00
BAL: $17,600.00


Date: 2025-08-03
Description: Pension Deposit - Demo Retirement Fund
Ref: PN445566
AMT: +$2,400.00
BAL: $20,000.00


Date: 2025-08-05
Description: Online Payment - Demo Mortgage Co
Ref: OP112233
AMT: -$1,245.00
BAL: $18,755.00


Date: 2025-08-07
Description: Card Purchase - Demo Medical Center
Ref: CP556677
AMT: -$125.50
BAL: $18,629.50


Date: 2025-08-10
Description: Card Purchase - Demo Hardware Store
Ref: CP778899
AMT: -$89.75
BAL: $18,539.75


Date: 2025-08-12
Description: Check Payment - Demo Insurance Co
Ref: CK334455
AMT: -$385.00
BAL: $18,154.75


Date: 2025-08-15
Description: Card Purchase - Demo Grocery Chain
Ref: CP889900
AMT: -$156.89
BAL: $17,997.86


Date: 2025-08-18
Description: Online Payment - Demo Electric Company
Ref: OP667788
AMT: -$145.75
BAL: $17,852.11


Date: 2025-08-20
Description: 